# Testing of hists/dataframe.py

In [1]:
%load_ext autoreload
%autoreload 2
from functools import partial
import pandas as pd
import numpy as np
import uproot
import awkward as ak
import sys
sys.path.append("..")
from hists.dataframe import *
from hists.shower_axis import *

In [2]:
#input_file = '/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/ClusteringAnalysis/CLUE_clusters_single.root'
input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v41/cmssw/sim_proton_v46_patchMIP/CLUE_clusters.root'
#input_file = '/data_cms_upgrade/cuisset/testbeam18/clue3d/v20/cmssw/sim_proton_v7/CLUE_clusters.root'

In [3]:
for array_i in uproot.iterate(input_file + ":clusters", step_size="10MB", library="ak"):
    series = array_i
    break

In [4]:
comp = DataframeComputations(series)

In [5]:
comp.rechits_totalReconstructedEnergyPerEventLayer.index.levels[1].to_numpy(dtype=np.int32)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int32)

In [7]:
comp.rechits_intervalHoldingFractionOfEnergy_joined(0.68)

,intervalFractionEnergy_minLayer,intervalFractionEnergy_maxLayer,beamEnergy,synchrotronBeamEnergy,intervalFractionEnergy_length
eventInternal,,,,,
0,4,12,100.0,99.83,9
1,6,13,100.0,99.83,8
2,4,12,100.0,99.83,9
3,5,12,100.0,99.83,8
4,5,12,100.0,99.83,8
...,...,...,...,...,...
1051,7,14,100.0,99.83,8
1052,8,17,100.0,99.83,10
1053,3,11,100.0,99.83,9


In [ ]:
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 500)

In [ ]:
comp_n = DataframeComputations(series)
comp_e = DataframeComputations(series[0:0])

In [ ]:
self = comp_e
self.clusters3D_merged_2D_custom(self.clusters3D_with_clus2D_id(), self.clusters2D)

In [ ]:
comp_e.clusters3D_with_clus2D_id().reset_index()

In [ ]:
self = comp_n
self.clusters3D_merged_2D_custom(self.clusters3D_with_clus2D_id(), self.clusters2D)

In [ ]:
df = comp.clusters3D_intervalHoldingFractionOfEnergy_joined(fraction=0.7)
df = df[df.index.isin(comp.clusters3D_largestClusterIndex_fast)]

In [ ]:
df

In [ ]:
comp.clusters3D_layerWithMaxClusteredEnergy

In [ ]:
pd.merge(df, 
    comp.clusters3D_layerWithMaxClusteredEnergy[["eventInternal","clus3D_id","layer_with_max_clustered_energy", "clus2D_energy_sum"]].set_index(["eventInternal", "clus3D_id"]), 
    left_index=True, right_index=True)

In [ ]:
array = comp.array
array

In [ ]:
array = array[ak.num(array.clus3D_energy, axis=-1) >= 2]
        # compute indices of 3D clusters for each event, sorted by energy
clus3D_energySortedIndices = ak.argsort(array['clus3D_energy'], axis=-1, ascending=False)#[:, 0]
clus3D_energySortedIndices[:, 0:1]

In [ ]:
clus2D_ids = array.clus3D_idxs[clus3D_energySortedIndices[:, 0:1]][:, 0, :]
clus2D_ids

In [ ]:
rechits_ids = ak.flatten(array.clus2D_idxs[array.clus3D_idxs[clus3D_energySortedIndices][:, 0, :]], axis=-1)
rechits_ids

In [ ]:
array[["rechits_x", "rechits_y"]][rechits_ids]

In [ ]:
for evt in zip(array.rechits_energy[rechits_ids], array.rechits_x[rechits_ids]):
    print(evt)